# Implementação no Python

## Importação de pacotes

In [14]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

## Fazendo uma requisição

In [10]:
def verify_status_code(url):
    response = requests.get(url)

    if response.status_code == 200:
        print(f'{url} status - ok')
        return response

    print(f'A requisição falhou. Status code {response.status_code}')
    return 0


urls = ['https://g1.globo.com/rss/g1/educacao/',
        'https://noticias.uol.com.br/sitemap/v2/today.xml',
        'https://www.r7.com/arc/outboundfeeds/sitemap-news/?outputType=xml']

for url in urls:
    response = verify_status_code(url)

https://g1.globo.com/rss/g1/educacao/ status - ok
 https://noticias.uol.com.br/sitemap/v2/today.xml status - ok
https://www.r7.com/arc/outboundfeeds/sitemap-news/?outputType=xml status - ok


### Raspando dados - `noticias.uol.com.br/sitemap/v2/today.xml`

In [24]:
target_url = 'https://noticias.uol.com.br/sitemap/v2/today.xml'
response = requests.get(target_url)  # coletando o xml do uol

soup = bs(response.content, "xml")

urls_data = [
    {
        "url": url.loc.text,
        "title": "",
        "subtitle": "",
        "ISO_datetime": url.lastmod.text
    }
for url in soup.find_all("url")]

for url in urls_data:

[{'url': 'https://noticias.uol.com.br/ultimas-noticias/agencia-brasil/2025/06/25/policia-civil-do-rio-combate-quadrilha-que-aplicava-golpes-em-idosos.htm',
  'title': '',
  'subtitle': '',
  'ISO_datetime': '2025-06-26T00:16:02.000Z'},
 {'url': 'https://noticias.uol.com.br/ultimas-noticias/agencia-estado/2025/06/25/bolsonaro-cita-eleicao-de-2026-para-criticar-julgamento-das-big-techs-no-stf.htm',
  'title': '',
  'subtitle': '',
  'ISO_datetime': '2025-06-26T00:12:07.000Z'},
 {'url': 'https://noticias.uol.com.br/ultimas-noticias/afp/2025/06/25/trump-pede-que-julgamento-de-netanyahu-por-corrupcao-seja-cancelado.htm',
  'title': '',
  'subtitle': '',
  'ISO_datetime': '2025-06-26T00:06:17.000Z'},
 {'url': 'https://noticias.uol.com.br/ultimas-noticias/2025/06/25/lotofacil-concurso-3426-sorteio-25-de-junho.htm',
  'title': '',
  'subtitle': '',
  'ISO_datetime': '2025-06-26T00:06:01.000Z'},
 {'url': 'https://noticias.uol.com.br/politica/ultimas-noticias/2025/06/25/carla-araujo-motta-surpre